In [ ]:
# =====================================================================================
#   DU AI CHALLENGE: FIXED ENSEMBLE TRAFFIC DETECTION MODEL
# =====================================================================================

# Cell 1: Setup and Installation
print("--- Step 1: Installing and Importing Libraries ---")
!pip install -q ultralytics pandas pyyaml tqdm opencv-python

import os
import yaml
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
import random
import shutil
from tqdm import tqdm
import cv2
import numpy as np

print("Setup complete. Libraries installed and imported.\n")


# Cell 2: Enhanced Configuration for Ensemble
print("--- Step 2: Setting Up Enhanced Configuration for Ensemble ---")

class Config:
    # --- Paths and Directories ---
    COMPETITION_DATA_DIR = 'du-ai-challenge'

    # --- Data Preparation (Same for all models) ---
    VALIDATION_SPLIT = 0.25
    USE_CLAHE_PREPROCESSING = True
    CLAHE_CLIP_LIMIT = 3.0
    CLAHE_TILE_GRID_SIZE = (8, 8)

    # --- ENSEMBLE MODEL CONFIGURATION ---
    ENSEMBLE_MODELS = {
        'yolov8l': 'yolov8l.pt',
        'yolov8m': 'yolov8m.pt',
        'yolov5l': 'yolov5l.pt',
    }
    
    # --- Training Parameters (Shared across models) ---
    TRAINING_EPOCHS = 100
    IMAGE_SIZE = 1280
    BATCH_SIZE = 4
    PATIENCE = 30
    LEARNING_RATE = 0.001
    FINAL_LR = 0.0001
    WARMUP_EPOCHS = 5
    OPTIMIZER = 'AdamW'
    WEIGHT_DECAY = 0.0005
    MOMENTUM = 0.937
    
    # --- Augmentation Strategy ---
    AUGMENTATION = {
        'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4,
        'degrees': 10.0, 'translate': 0.1, 'scale': 0.5, 'shear': 1.0,
        'perspective': 0.0005, 'flipud': 0.0, 'fliplr': 0.5,
        'mosaic': 1.0, 'mixup': 0.05, 'copy_paste': 0.05,
        'dropout': 0.1, 'erasing': 0.2,
    }
    CLOSE_MOSAIC_EPOCHS = 20
    
    # --- Loss Function Weights ---
    BOX_GAIN = 0.05
    CLS_GAIN = 0.5
    DFL_GAIN = 1.5

    # --- INFERENCE PARAMETERS FOR ENSEMBLE ---
    CONFIDENCE_THRESHOLD = 0.1
    ENSEMBLE_IOU_THRESH = 0.6
    MODEL_IOU_THRESH = 0.7

# Setup paths
Config.KAGGLE_INPUT_PATH = f'/kaggle/input/{Config.COMPETITION_DATA_DIR}'
Config.WORKING_DIR = '/kaggle/working/'

if not os.path.exists(Config.KAGGLE_INPUT_PATH):
    raise FileNotFoundError(f"Data not found at {Config.KAGGLE_INPUT_PATH}")

print("Ensemble models to be trained:", list(Config.ENSEMBLE_MODELS.keys()))
print(f"CLAHE Preprocessing: {Config.USE_CLAHE_PREPROCESSING}\n")


# Cell 3: Enhanced Data Preparation
def preprocess_with_clahe(input_dir, output_dir):
    """Enhanced CLAHE preprocessing with better parameters."""
    clahe = cv2.createCLAHE(
        clipLimit=Config.CLAHE_CLIP_LIMIT, 
        tileGridSize=Config.CLAHE_TILE_GRID_SIZE
    )
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in tqdm(os.listdir(input_dir), desc=f"CLAHE on {os.path.basename(input_dir)}"):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
            
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None: 
            continue
            
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        l_clahe = clahe.apply(l)
        lab_clahe = cv2.merge((l_clahe, a, b))
        final_img = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
        
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        final_img = cv2.filter2D(final_img, -1, kernel * 0.1) + final_img * 0.9
        final_img = np.clip(final_img, 0, 255).astype(np.uint8)
        
        cv2.imwrite(os.path.join(output_dir, filename), final_img)

def prepare_enhanced_data_and_yaml():
    """Enhanced data preparation with proper validation split."""
    print("--- Step 3: Enhanced Data Preparation (Once for all models) ---")
    
    source_images_path = os.path.join(Config.KAGGLE_INPUT_PATH, 'train/images')
    source_labels_path = os.path.join(Config.KAGGLE_INPUT_PATH, 'train/labels')
    
    final_dataset_dir = os.path.join(Config.WORKING_DIR, 'dataset')
    train_img_path = os.path.join(final_dataset_dir, 'train/images')
    train_lbl_path = os.path.join(final_dataset_dir, 'train/labels')
    val_img_path = os.path.join(final_dataset_dir, 'valid/images')
    val_lbl_path = os.path.join(final_dataset_dir, 'valid/labels')
    test_img_path = os.path.join(final_dataset_dir, 'test/images')
    
    for p in [train_img_path, train_lbl_path, val_img_path, val_lbl_path, test_img_path]:
        os.makedirs(p, exist_ok=True)
    
    all_image_files = sorted([f for f in os.listdir(source_images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    random.seed(42)
    random.shuffle(all_image_files)
    
    split_idx = int(len(all_image_files) * (1 - Config.VALIDATION_SPLIT))
    train_files, val_files = all_image_files[:split_idx], all_image_files[split_idx:]
    
    print(f"Train files: {len(train_files)}, Validation files: {len(val_files)}")
    
    for f_list, lbl_path in [(train_files, train_lbl_path), (val_files, val_lbl_path)]:
        for f in f_list:
            label_file = Path(f).stem + '.txt'
            src_label = os.path.join(source_labels_path, label_file)
            if os.path.exists(src_label):
                shutil.copy(src_label, lbl_path)
    
    if Config.USE_CLAHE_PREPROCESSING:
        print("Applying enhanced CLAHE preprocessing...")
        temp_train_img_dir, temp_val_img_dir = '/tmp/train_images/', '/tmp/val_images/'
        os.makedirs(temp_train_img_dir, exist_ok=True); os.makedirs(temp_val_img_dir, exist_ok=True)
        for f in train_files: shutil.copy(os.path.join(source_images_path, f), temp_train_img_dir)
        for f in val_files: shutil.copy(os.path.join(source_images_path, f), temp_val_img_dir)
        
        preprocess_with_clahe(temp_train_img_dir, train_img_path)
        preprocess_with_clahe(temp_val_img_dir, val_img_path)
        preprocess_with_clahe(os.path.join(Config.KAGGLE_INPUT_PATH, 'test/images'), test_img_path)
        
        shutil.rmtree(temp_train_img_dir); shutil.rmtree(temp_val_img_dir)
    else:
        for f in train_files: shutil.copy(os.path.join(source_images_path, f), train_img_path)
        for f in val_files: shutil.copy(os.path.join(source_images_path, f), val_img_path)
        shutil.copytree(os.path.join(Config.KAGGLE_INPUT_PATH, 'test/images'), test_img_path, dirs_exist_ok=True)
        
    class_names_path = os.path.join(Config.KAGGLE_INPUT_PATH, 'class_names.txt')
    with open(class_names_path, 'r') as f:
        class_names = [line.strip().split(': ')[1] for line in f if ':' in line]

    data_yaml_content = {
        'path': os.path.abspath(final_dataset_dir),
        'train': 'train/images', 'val': 'valid/images', 'test': 'test/images',
        'nc': len(class_names), 'names': class_names
    }
    
    yaml_path = os.path.join(Config.WORKING_DIR, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml_content, f, default_flow_style=False)
        
    print(f"Enhanced data.yaml created at {yaml_path}\n")
    return yaml_path


# Cell 4: Train Ensemble Models 
def train_ensemble_models(data_yaml_path):
    """Trains each model using the unified ultralytics API."""
    print("--- Step 4: Training Ensemble Models ---")
    trained_model_paths = {}

    training_args = {
        'data': data_yaml_path, 'epochs': Config.TRAINING_EPOCHS, 'imgsz': Config.IMAGE_SIZE,
        'batch': Config.BATCH_SIZE, 'patience': Config.PATIENCE, 'optimizer': Config.OPTIMIZER,
        'lr0': Config.LEARNING_RATE, 'lrf': Config.FINAL_LR, 'momentum': Config.MOMENTUM,
        'weight_decay': Config.WEIGHT_DECAY, 'warmup_epochs': Config.WARMUP_EPOCHS,
        'cos_lr': True, 'box': Config.BOX_GAIN, 'cls': Config.CLS_GAIN, 'dfl': Config.DFL_GAIN,
        'close_mosaic': Config.CLOSE_MOSAIC_EPOCHS, 'project': Config.WORKING_DIR,
        'exist_ok': True, 'seed': 42, 'device': 0, 'save_period': 10, 'amp': True,
        **Config.AUGMENTATION,
    }
    
    for model_name, model_file in Config.ENSEMBLE_MODELS.items():
        print(f"\n--- Training Model: {model_name} ({model_file}) ---")
        model = YOLO(model_file)
        run_name = f'training_run_{model_name}'
        model.train(**training_args, name=run_name)
        
        best_model_path = os.path.join(Config.WORKING_DIR, run_name, 'weights/best.pt')
        if os.path.exists(best_model_path):
            trained_model_paths[model_name] = best_model_path
            print(f"✅ Training complete for {model_name}. Best model: {best_model_path}")
        else:
            raise FileNotFoundError(f"Could not find best.pt for {model_name} after training.")

    return trained_model_paths


# Cell 5: Ensemble Inference and Submission Generation 
def apply_ensemble_nms(all_boxes, iou_threshold):
    """Applies Non-Maximum Suppression to a list of boxes from multiple models."""
    if not all_boxes: return []
    class_groups = {}
    for box in all_boxes:
        class_id = int(box[5])
        if class_id not in class_groups: class_groups[class_id] = []
        class_groups[class_id].append(box)
    final_boxes = []
    for class_id, boxes in class_groups.items():
        boxes.sort(key=lambda x: x[4], reverse=True)
        kept_boxes = []
        while boxes:
            best_box = boxes.pop(0)
            kept_boxes.append(best_box)
            remaining_boxes = []
            for other_box in boxes:
                x1_inter = max(best_box[0], other_box[0]); y1_inter = max(best_box[1], other_box[1])
                x2_inter = min(best_box[2], other_box[2]); y2_inter = min(best_box[3], other_box[3])
                inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
                if inter_area == 0:
                    remaining_boxes.append(other_box); continue
                best_area = (best_box[2]-best_box[0])*(best_box[3]-best_box[1])
                other_area = (other_box[2]-other_box[0])*(other_box[3]-other_box[1])
                iou = inter_area / (best_area + other_area - inter_area)
                if iou < iou_threshold: remaining_boxes.append(other_box)
            boxes = remaining_boxes
        final_boxes.extend(kept_boxes)
    return final_boxes

def generate_ensemble_submission(model_paths):
    """Generates submission by running all models and ensembling their predictions."""
    print("\n--- Step 5: Generating Submission from Ensemble ---")
    models = {name: YOLO(path) for name, path in model_paths.items()}
    print("Ensemble models loaded:", list(models.keys()))

    test_images_dir = os.path.join(Config.WORKING_DIR, 'dataset/test/images')
    test_image_paths = sorted(list(Path(test_images_dir).glob('*.jpg')))
    print(f"Found {len(test_image_paths)} test images")

    all_predictions = []
    for img_path in tqdm(test_image_paths, desc="Ensemble Inference"):
        all_image_boxes = []
        for model_name, model in models.items():
            results = model.predict(source=str(img_path), conf=Config.CONFIDENCE_THRESHOLD, iou=Config.MODEL_IOU_THRESH, imgsz=Config.IMAGE_SIZE, agnostic_nms=True, max_det=300, verbose=False)
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    all_image_boxes.append([x1, y1, x2, y2, box.conf[0].item(), box.cls[0].item()])
        
        final_boxes = apply_ensemble_nms(all_image_boxes, Config.ENSEMBLE_IOU_THRESH)
        
        prediction_strings = [f"{int(cls)} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}" for x1, y1, x2, y2, conf, cls in final_boxes]
        full_prediction_string = " ".join(prediction_strings)
        image_id = Path(img_path.name).stem
        
        all_predictions.append({'image_id': image_id, 'PredictionString': full_prediction_string})

    submission_df = pd.DataFrame(all_predictions)
    submission_path = os.path.join(Config.WORKING_DIR, 'ensemble_submission.csv')
    submission_df.to_csv(submission_path, index=False)

    print(f"\nEnsemble submission created: {submission_path}")
    print("--- Submission Preview ---\n", submission_df.head())
    non_empty_preds = sum(1 for pred in all_predictions if pred['PredictionString'].strip())
    print(f"\nStatistics:\n  Total images: {len(all_predictions)}")
    print(f"  Images with detections: {non_empty_preds} ({non_empty_preds/len(all_predictions)*100:.1f}%)")
    return submission_path


# Cell 6: Run Ensemble Pipeline
if __name__ == '__main__':
    print("Starting Fixed Ensemble Traffic Detection Pipeline\n")
    try:
        yaml_file = prepare_enhanced_data_and_yaml()
        trained_models = train_ensemble_models(yaml_file)
        submission_file = generate_ensemble_submission(trained_models)
        
        print(f"\n🎉 Ensemble Pipeline completed successfully!")
        print("📁 Best trained models:")
        for name, path in trained_models.items():
            print(f"  - {name}: {path}")
        print(f"📄 Submission file ready: {submission_file}")
        
    except Exception as e:
        import traceback
        print(f"\n❌ Pipeline failed with error: {str(e)}")
        traceback.print_exc()

--- Step 1: Installing and Importing Libraries ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings

CLAHE on images: 100%|██████████| 57/57 [00:28<00:00,  2.01it/s]


Enhanced data.yaml created at /kaggle/working/data.yaml

--- Step 4: Training Ensemble Models ---

--- Training Model: yolov8l (yolov8l.pt) ---


Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=0.05, cache=False, cfg=None, classes=None, close_mosaic=20, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.2, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=training_run_yolov8l, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=30, perspective=0.0005, plots=True, pos

Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytic

...


98 epochs completed in 0.801 hours.
Optimizer stripped from /kaggle/working/training_run_yolov5l/weights/last.pt, 106.9MB
Optimizer stripped from /kaggle/working/training_run_yolov5l/weights/best.pt, 106.9MB

Validating /kaggle/working/training_run_yolov5l/weights/best.pt...
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5l summary (fused): 128 layers, 53,139,889 parameters, 0 gradients, 134.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.86it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         44       1314      0.712      0.643      0.677      0.345
               bicycle         15         17      0.785      0.645      0.721      0.222
                   bus          8         32      0.674      0.562      0.556      0.341
                   car         36        234      0.739      0.701      0.774      0.411
                   cng         31        109      0.806      0.798      0.835      0.462
                leguna          3          9          1      0.452       0.67      0.396
            manual-van         17         35      0.761      0.714      0.761       0.38
                 motor         34         92      0.652      0.694      0.689      0.345
                others          7         12      0.588      0.917      0.784      0.507
            pedestrian         41        313      0.673      0.498      0.548       0.21
              rickshaw         41        439        0.7      0.777      0.774       0.38
                 truc

Ensemble Inference: 100%|██████████| 57/57 [00:24<00:00,  2.33it/s]


Ensemble submission created: /kaggle/working/ensemble_submission.csv
--- Submission Preview ---
                                             image_id  \
0  DJI_0017_000057_jpg.rf.a2ce1add5011cc942c1c150...   
1  DJI_0018_000001_jpg.rf.e11080fe695e8a6f1c4bee9...   
2  DJI_0018_000003_jpg.rf.4d6950f51b4bda0e8b884bf...   
3  DJI_0019_000001_jpg.rf.e0d9ea54ee3f8606ae2897d...   
4  DJI_0019_000012_jpg.rf.d12efa280da8f6e25207a2c...   

                                    PredictionString  
0  9 0.8696 0 1112 231 1594 9 0.8216 238 1111 521...  
1  2 0.9065 723 1729 1587 2047 2 0.8819 1625 1676...  
2  2 0.9161 733 1735 1597 2069 2 0.9016 1642 1673...  
3  2 0.9089 488 1601 1367 1915 2 0.8857 1413 1555...  
4  6 0.8722 1366 1647 1671 1920 6 0.1169 3870 140...  

Statistics:
  Total images: 57
  Images with detections: 57 (100.0%)

🎉 Ensemble Pipeline completed successfully!
📁 Best trained models:
  - yolov8l: /kaggle/working/training_run_yolov8l/weights/best.pt
  - yolov8m: /kaggle/working/tr